In [24]:
!pip install xgboost lightgbm

You should consider upgrading via the 'f:\syu\learning\rikkyo-openhousecomp\.venv\scripts\python.exe -m pip install --upgrade pip' command.


In [25]:
!pip install imbalanced-learn

You should consider upgrading via the 'f:\syu\learning\rikkyo-openhousecomp\.venv\scripts\python.exe -m pip install --upgrade pip' command.


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [27]:
#TRAIN_DATA_PATH = "/content/drive/MyDrive/機械学習コンペ/dataset/train_Mod.csv"
TRAIN_DATA_PATH = "dataset/train_Mod.csv"
df = pd.read_csv(TRAIN_DATA_PATH)

# 外れ値の除去
# 外れ値を削除
df_ = df[df['knee_depth'] < 17.5]
df_ = df_[df_['jump_height'] < 80]

# 欠損値を持つ行を削除
df2 = df_.dropna()

# 説明変数と目的変数を分割
X = df2.iloc[:, 1:6]
y = df2.iloc[:, 6]

# 訓練データと検証データを分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(y_train.value_counts())
# one-hotエンコーディング
y_train_onehot = pd.get_dummies(y_train)
y_val_onehot = pd.get_dummies(y_val)


# SMOTEを使用してデータを水増し
smote = SMOTE(random_state=123, sampling_strategy={'A': 700, 'B': 700, 'C': 700, 'D': 700, 'E': 700, 'F':700})
X_train, y_train = smote.fit_resample(X_train, y_train)
print(X_train.shape, y_train.shape)

# one-hotエンコーディングを再度適用
y_train_onehot = pd.get_dummies(y_train)
# 分布の確認
print(y_train.value_counts())

"""
#相関係数が高いもの(knee_depth, hip_depth)を削除
X_train = X_train.drop(['hip_depth'], axis=1)
X_val = X_val.drop(['hip_depth'], axis=1)
"""

(483, 5) (483,) (121, 5) (121,)
personal_id
D    105
A     89
B     89
C     80
E     60
F     60
Name: count, dtype: int64
(4200, 5) (4200,)
personal_id
E    700
D    700
C    700
F    700
A    700
B    700
Name: count, dtype: int64


"\n#相関係数が高いもの(knee_depth, hip_depth)を削除\nX_train = X_train.drop(['hip_depth'], axis=1)\nX_val = X_val.drop(['hip_depth'], axis=1)\n"

In [28]:
X_train.describe()

,knee_depth,hip_depth,misalignment,width_of_hand,jump_height
count,4200.000000,4200.000000,4200.000000,4200.000000,4200.000000
mean,8.140685,24.122637,4.888521,-12.107412,35.885102
std,2.502427,9.687946,5.781007,23.695810,7.060138
min,0.183013,0.002281,-17.492876,-100.379578,15.982949
25%,5.965050,15.212567,0.883467,-25.891450,31.866314
50%,8.694794,25.899886,4.261980,-8.844954,34.766658
75%,9.855261,31.542589,8.787414,1.290427,38.904162
max,14.617088,46.143636,26.511442,45.934550,67.422717


In [29]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

#最適なパラメータで保存
param = {
    'classifier__colsample_bytree': [1.0],
    'classifier__gamma': [0.2],
    'classifier__learning_rate': [0.3],
    'classifier__max_depth': [5],
    'classifier__min_child_weight': [1],
    'classifier__n_estimators': [50],
    'classifier__subsample': [0.9]}
pipeline = Pipeline([
    ('classifier', XGBClassifier(eval_metric='mlogloss'))  # ステップ3: XGBoost
])


model = GridSearchCV(pipeline, param, cv=5, scoring='accuracy', n_jobs=-1)
model.fit(X_train, y_train_onehot)

#最適なパラメータを表示
print("the best param is : ", model.best_params_)
print("the best score is : ", model.best_score_)
#検証データで評価
y_pred = model.predict(X_val)
print("valid accuracy_score : ", accuracy_score(y_val_onehot, y_pred))

best_xgb = model.best_estimator_

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

the best param is :  {'classifier__colsample_bytree': 1.0, 'classifier__gamma': 0.2, 'classifier__learning_rate': 0.3, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 50, 'classifier__subsample': 0.9}
the best score is :  0.8411904761904762
valid accuracy_score :  0.7107438016528925


f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [30]:
import lightgbm as lgb

param = {
    'learning_rate': [0.1], 'max_depth': [5], 'n_estimators': [50]
}

# LightGBMのモデルを作成
lgbm = lgb.LGBMClassifier(objective='multiclass', num_class=6)

# GridSearchCVの設定
grid_search = GridSearchCV(lgbm, param, cv=3)

# ハイパーパラメータ探索
grid_search.fit(X_train, y_train)

# 最適なパラメータとその時のスコアを表示
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

# 最適なパラメータでモデルを訓練
best_lgbm = grid_search.best_estimator_

# テストデータで評価
y_pred = best_lgbm.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Valid accuracy: {accuracy}")

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000135 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2800, number of used features: 5
[LightGBM] [Info] Start training from score -1.791045
[LightGBM] [Info] Start training from score -1.791045
[LightGBM] [Info] Start training from score -1.791045
[LightGBM] [Info] Start training from score -1.791045
[LightGBM] [Info] Start training from score -1.793189
[LightGBM] [Info] Start training from score -1.793189
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [ ]:
#ランダムフォレスト


In [31]:
# アンサンブル学習の適用

from sklearn.ensemble import VotingClassifier

# アンサンブルモデルの設定（Voting Classifier）
ensemble_model = VotingClassifier(estimators=[('xgb', best_xgb), ('lgb', best_lgbm)], voting='soft')
# モデルの訓練
ensemble_model.fit(X_train, y_train)

# テストデータで評価
y_pred = ensemble_model.predict(X_val)
val_score = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred , average="micro")
print(f"Valid accuracy: {val_score}")
print(f"F1 score: {f1}")

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 4200, number of used features: 5
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

In [39]:
test_data = pd.DataFrame(pd.read_csv("dataset/test.csv"))
#idを保持
test_id = test_data['id']
test_data = test_data.drop(['id'], axis=1)
test_data = test_data.drop(['personal_id'], axis=1)
test_data.describe()

,knee_depth,hip_depth,misalignment,width_of_hand,jump_height
count,120.000000,120.000000,120.000000,120.000000,120.000000
mean,8.141072,23.829561,4.716138,-10.447167,36.213289
std,2.565012,10.133983,5.816611,22.616819,9.754696
min,1.617048,1.822088,-9.076842,-82.861069,19.632742
25%,6.318506,14.536246,0.525725,-21.442122,31.895944
50%,8.534731,25.566689,4.289263,-8.381027,34.870439
75%,9.997241,31.292131,7.966704,2.991329,38.446580
max,13.300155,45.552326,22.965621,40.637361,107.726423


In [41]:
#テストデータで評価

y_test_pred = ensemble_model.predict(test_data)

#提出用ファイルの作成(idとpersonal_id（モデルの予測値）のみ保持)
submission = pd.DataFrame({'id': test_id, 'personal_id': y_test_pred})
submission.to_csv('submission2.csv', index=True)




[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
f:\syu\Learning\Rikkyo-OpenHouseComp\.venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead